# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [39]:
import pandas as pd
import requests

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

# Requisição à API
response = requests.get(url)

# Verificar se a requisição foi bem sucedida e puxar as colunas da base de dados
if response.status_code == 200:
    data = response.json()
    # Converter os dados em DataFrame
    df = pd.DataFrame(data['items'])
    print("Dados carregados com sucesso!")
    # Exibir as colunas do DataFrame
    print("Colunas do DataFrame:", df.columns.tolist())
else:
    print(f"Erro ao carregar os dados: {response.status_code}")

Dados carregados com sucesso!
Colunas do DataFrame: ['id_pleito', 'tipo_interessado', 'interessado', 'cod_ibge', 'uf', 'num_pvl', 'status', 'num_processo', 'data_protocolo', 'tipo_operacao', 'finalidade', 'tipo_credor', 'credor', 'moeda', 'valor', 'pvl_assoc_divida', 'pvl_contradado_credor', 'data_status']


####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [41]:
# 1) Seu código aqui

# Filtrar os PVLs realizados por municípios no estado do Rio de Janeiro
df_rj = df[df['uf'] == 'RJ']
print(df_rj.head())

     id_pleito tipo_interessado          interessado  cod_ibge  uf  \
66       21870        Município       Rio de Janeiro   3304557  RJ   
92       63831        Município             Mesquita   3302858  RJ   
116      68563        Município          Nova Iguaçu   3303500  RJ   
178       9297        Município          Nova Iguaçu   3303500  RJ   
267      66228        Município  São Pedro da Aldeia   3305208  RJ   

                  num_pvl                           status  \
66   00000.000000/2028-07                         Deferido   
92   PVL02.000044/2023-14                         Deferido   
116  PVL02.000551/2024-39  Em retificação pelo interessado   
178                  None                     Regularizado   
267  PVL02.003177/2023-42                Deferido (PVL-IF)   

             num_processo        data_protocolo  \
66   17944.000312/2016-31  2016-12-21T02:00:00Z   
92   17944.101173/2023-91  2023-03-30T14:02:21Z   
116  17944.002171/2024-00  2024-05-28T20:07:11Z   
17

In [45]:
# 2) Seu código aqui
status_frequencia = df_rj['status'].value_counts()

# Obter os três status mais frequentes
top_3_status = status_frequencia.head(3)
print("Os três status mais frequentes são:")
print(top_3_status)

Os três status mais frequentes são:
status
Deferido                                                 26
Arquivado                                                11
Encaminhado à PGFN com manifestação técnica favorável    11
Name: count, dtype: int64


In [109]:
# 3) Seu código aqui
# Verificar o formato da coluna 'data_status'
print(df_rj['data_status'].head())

# Convertendo a coluna 'data_status' para datetime
df_rj['data_status'] = pd.to_datetime(df_rj['data_status'], format='%d/%m/%Y', errors='coerce')

# Extrair o ano da coluna 'data_status'
df_rj.loc[:, 'ano_status'] = df_rj['data_status'].dt.year
print(df_rj[['data_status', 'ano_status']].head())

66    2016-12-28
92    2023-03-31
116   2024-06-14
178   2007-08-02
267   2023-09-12
Name: data_status, dtype: datetime64[ns]
    data_status  ano_status
66   2016-12-28        2016
92   2023-03-31        2023
116  2024-06-14        2024
178  2007-08-02        2007
267  2023-09-12        2023


C:\Users\suporte\AppData\Local\Temp\ipykernel_7452\3615261995.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_rj['data_status'] = pd.to_datetime(df_rj['data_status'], format='%d/%m/%Y', errors='coerce')


In [53]:
# Contar a frequência de cada ano do campo 'ano_status'
ano_frequencia = df_rj['ano_status'].value_counts()
print("Frequência de cada ano no campo 'ano_status':")
print(ano_frequencia)

Frequência de cada ano no campo 'ano_status':
ano_status
2007    8
2008    8
2014    7
2023    6
2011    5
2012    5
2009    4
2019    3
2016    2
2010    2
2017    2
2022    2
2015    2
2006    1
2003    1
2024    1
2004    1
2002    1
2013    1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

URL utilizado em aula: https://www.meilisearch.com/movies.json

Este URL é um endpoint de uma API que fornece uma base de dados sobre filmes.
O endpoint retorna dados no formato JSON.
É um exemplo típico de um API REST que retorna uma coleção de recursos.

URL da tarefa: https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl

Este URL é um endpoint de uma API que fornece dados sobre PVLs (Pedidos de Verificação de Limites) realizados por municípios e estados.
O endpoint também retorna dados no formato JSON.

Este URL faz parte de um sistema mais complexo que pode envolver múltiplas tabelas e relacionamentos entre dados.
Ambos os URLs utilizam o protocolo HTTP e retornam dados no formato JSON, mas servem propósitos diferentes. O primeiro fornece dados sobre filmes, enquanto o segundo fornece dados administrativos sobre solicitações de verificação de limites financeiros por estados e municípios.

In [55]:
#1) Seu código aqui

# Importando as bibliotecas
import pandas as pd
import requests

def consultar_pvl(uf, tipo_interessado):
    """
    Consulta a API de PVLs com base no UF e no tipo de interessado.
    
    Args:
    uf (str): Unidade Federativa para a consulta.
    tipo_interessado (str): Tipo de interessado ('Estado' ou 'Município').
    
    Returns:
    DataFrame: Dados da consulta no formato DataFrame.
    """
    # URL da API
    url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    
    # Requisição à API
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['items'])
        
        # Filtrar dados com base no UF e tipo de interessado
        df_filtered = df[(df['uf'] == uf) & (df['interessado'] == tipo_interessado)]
        
        return df_filtered
    else:
        print(f"Erro ao carregar os dados: {response.status_code}")
        return pd.DataFrame()

# Testando a função
df_mg_estado = consultar_pvl('MG', 'Estado')
print(df_mg_estado.head())


Empty DataFrame
Columns: [id_pleito, tipo_interessado, interessado, cod_ibge, uf, num_pvl, status, num_processo, data_protocolo, tipo_operacao, finalidade, tipo_credor, credor, moeda, valor, pvl_assoc_divida, pvl_contradado_credor, data_status]
Index: []


In [73]:
# 2) Seu código aqui
# Consultar dados do Estado de Minas Gerais
if 'status' in df_mg_estado.columns:
    # Filtrar dados com status 'Arquivado por decurso de prazo'
    arquivados_mg = df_mg_estado[df_mg_estado['status'] == 'Arquivado por decurso de prazo']

    # Contar o número de solicitações arquivadas
    num_arquivados_mg = arquivados_mg.shape[0]
    print(f"Número de solicitações arquivadas por decurso de prazo em Minas Gerais: {num_arquivados_mg}")

Número de solicitações arquivadas por decurso de prazo em Minas Gerais: 0


In [101]:
# 3) Seu código aqui
# Contar as solicitações deferidas da Bahia
if not deferidos_ba.empty:
    municipio_mais_deferidos = deferidos_ba['interessado'].value_counts().idxmax()
    print(f"Município da Bahia com mais solicitações deferidas: {municipio_mais_deferidos}")
else:
    print("Não há solicitações deferidas na Bahia para municípios.")


Não há solicitações deferidas na Bahia para municípios.


In [105]:
# 4) Seu código aqui
# Salvar os dados em um arquivo CSV
df_ba_estado.to_csv('solicitacoes_ba_estado.csv', index=False)
print("Dados de solicitações da Bahia com interessado 'Estado' foram salvos em 'solicitacoes_ba_estado.csv'.")

Dados de solicitações da Bahia com interessado 'Estado' foram salvos em 'solicitacoes_ba_estado.csv'.
